## Notebook 02: Tiền xử lý dữ liệu (từ kết quả EDA)

### Quy trình tổng quát
1. **Nạp dữ liệu thô**
2. **Preprocessing**: Kiểm tra
    - Dữ liệu có thiếu không?
    - Các cột chính có hợp lệ không?
    - Kiểm tra outliers

### Ghi chú từ EDA
- Phân phối rating lệch nhưng vẫn nằm hoàn toàn trong [1, 5] và không có giá trị nhiễu.
- 14.7% đánh giá 1-2 sao mang ý nghĩa tiêu cực hợp lệ => giữ nguyên

In [1]:
import sys
import os
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from data_preprocessing import (
    load_data,
    run_validation_checks,
    run_outlier_checks,
    validate_data,
    save_processed_data
)

# Config
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
np.set_printoptions(precision=3, suppress=True)


## 1: Load raw data

In [2]:
# Đường dẫn dữ liệu
RAW_DATA_PATH = '../data/raw/ratings_Beauty.csv'
PROCESSED_DATA_PATH = '../data/processed/ratings_Beauty_processed'

print("=" * 70)
print("BƯỚC 1: NẠP DỮ LIỆU THÔ")
print("=" * 70)
print(f"\nĐang đọc: {RAW_DATA_PATH}")
print("(Notebook 01 xác nhận dữ liệu sạch nên có thể đọc trực tiếp)")

data = load_data(RAW_DATA_PATH)

print("\n Đọc dữ liệu thành công!")
print(f"  Kích thước: {data.shape}")
print(f"  Kiểu dữ liệu: {data.dtype}")

print("\n5 dòng đầu tiên:")
for i in range(min(5, len(data))):
    print(f"  {data[i]}")

BƯỚC 1: NẠP DỮ LIỆU THÔ

Đang đọc: ../data/raw/ratings_Beauty.csv
(Notebook 01 xác nhận dữ liệu sạch nên có thể đọc trực tiếp)
 Đọc dữ liệu thành công từ ../data/raw/ratings_Beauty.csv
  Số dòng: 2,023,070

 Đọc dữ liệu thành công!
  Kích thước: (2023070,)
  Kiểu dữ liệu: [('UserId', '<U50'), ('ProductId', '<U50'), ('Rating', '<f4'), ('Timestamp', '<i8')]

5 dòng đầu tiên:
  ('A39HTATAQ9V7YF', '0205616461', 5., 1369699200)
  ('A3JM6GV9MNOF9X', '0558925278', 3., 1355443200)
  ('A1Z513UWSAAO0F', '0558925278', 5., 1404691200)
  ('A1WMRR494NWEWV', '0733001998', 4., 1382572800)
  ('A3IAAVS479H7M7', '0737104473', 1., 1274227200)


## 2: Kiểm tra tính hợp lệ dữ liệu (Data Validation)

In [6]:
validation_stats = run_validation_checks(data)

BƯỚC 2: KIỂM TRA TÍNH HỢP LỆ DỮ LIỆU

=== KIỂM TRA MISSING VALUES ===
  UserId         : ✓ Đầy đủ     (0 missing)
  ProductId      : ✓ Đầy đủ     (0 missing)
  Rating         : ✓ Đầy đủ     (0 missing)
  Timestamp      : ✓ Đầy đủ     (0 missing)

  Tổng missing values: 0

[2.2] KIỂM TRA KHOẢNG GIÁ TRỊ HỢP LỆ
----------------------------------------------------------------------
Rating ngoài [1,5]: 0
  Rating 1.0: 183,784 (9.1%)
  Rating 2.0: 113,034 (5.6%)
  Rating 3.0: 169,791 (8.4%)
  Rating 4.0: 307,740 (15.2%)
  Rating 5.0: 1,248,721 (61.7%)

 PASS: Tất cả ratings hợp lệ [1-5]

[2.3] KIỂM TRA TRÙNG LẶP
----------------------------------------------------------------------
Users duy nhất: 1,210,271
Products duy nhất: 249,274
Sparsity: 100.00%

Cặp (user,product) trùng: 0 (0.00%)


In [4]:
outlier_report = run_outlier_checks(data)

BƯỚC 3: PHÁT HIỆN GIÁ TRỊ BẤT THƯỜNG

[USER BEHAVIOR OUTLIERS]
----------------------------------------------------------------------
Ratings/user behavior outliers - Min: 1, Max: 389
Ratings/user behavior outliers - Mean: 1.7, Median: 1

Outlier threshold (IQR): 4
USER BEHAVIOR OUTLIERS vượt ngưỡng: 82,659 (6.8%)

 Giữ nguyên: Power users có giá trị cho CF

[PRODUCT POPULARITY OUTLIERS]
----------------------------------------------------------------------
Ratings/product popularity outliers - Min: 1, Max: 7,533
Ratings/product popularity outliers - Mean: 8.1, Median: 2

Outlier threshold (IQR): 11
PRODUCT POPULARITY OUTLIERS vượt ngưỡng: 31,337 (12.6%)

 Giữ nguyên: Popular products có giá trị cho recommendation


**Nhận xét**: 
- Dataset này khá sạch và không có nhiều features (chỉ có 2 numerical features là Timestamp và Rating)
- Vì rating nằm trong đoạn [1-5] là xử lý được nên không cần thiết minmax hay log scaling

## 4: Lưu dữ liệu sạch ra `.npz`

Sau khi kiểm tra hợp lệ và outlier, chúng ta lưu toàn bộ dataset vào định dạng `.npz` để tái sử dụng ở bước modeling.


In [5]:
print("=" * 70)
print("BƯỚC 4: LƯU DỮ LIỆU SẠCH")
print("=" * 70)

CLEAN_OUTPUT_PATH = '../data/processed/ratings_Beauty_processed_clean'

# Dùng lại hàm validate_data để đảm bảo dữ liệu hợp lệ
clean_data, removed = validate_data(data)
print(f"\nSố dòng bị loại bỏ: {removed:,}")
print(f"Kích thước dữ liệu sạch: {len(clean_data):,}")

save_processed_data(clean_data, CLEAN_OUTPUT_PATH)
print(f"\nFile đã sẵn sàng: {CLEAN_OUTPUT_PATH}.npz")


BƯỚC 4: LƯU DỮ LIỆU SẠCH

=== KIỂM TRA TÍNH HỢP LỆ ===
  Số dòng ban đầu: 2,023,070
  Số dòng không hợp lệ: 0
  Số dòng còn lại: 2,023,070
  Tỷ lệ giữ lại: 100.00%

Số dòng bị loại bỏ: 0
Kích thước dữ liệu sạch: 2,023,070

=== LƯU DỮ LIỆU ===
  Saved: ../data/processed/ratings_Beauty_processed_clean.npz (compressed)
  Kích thước: 2,023,070 records
  File size: 52.29 MB

File đã sẵn sàng: ../data/processed/ratings_Beauty_processed_clean.npz
